In [13]:
import os
import shutil
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# -------------------------------------------------------------------------
# 1. 路径定义 (根据您的本地环境)
# -------------------------------------------------------------------------
BASE = r"D:\Columbia\Fall2025\5400\project\layer"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

# 各层级路径
BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
GOLD = os.path.join(BASE, "gold")
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
SENTIMENT_OUTPUT = os.path.join(BASE, "sentiment_output")       # 新增: NLP 输出
FACT_NEWS_SENTIMENT = os.path.join(BASE, "fact_news_sentiment") # 新增: 最终事实表

# -------------------------------------------------------------------------
# 2. Spark Session 初始化
# -------------------------------------------------------------------------
def create_spark(app_name="5400-news-elt-optimized"):
    # 注意：为了连接 Snowflake，需要加载对应的 Maven 包
    # 如果您本地网络无法自动下载 JAR，可能需要手动下载并配置 spark.jars
    snowflake_package = "net.snowflake:spark-snowflake_2.12:2.11.0-spark_3.3"

    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        # 添加 Snowflake 依赖 (根据您的 Spark 版本调整 Scala/Spark 版本号)
        .config("spark.jars.packages", snowflake_package)
        .getOrCreate()
    )
    return spark

In [14]:
def build_bronze(spark):
    """Raw CSV -> Parquet (Bronze Layer)"""
    print("🚀 Building Bronze Layer...")
    if os.path.exists(BRONZE): shutil.rmtree(BRONZE)

    df = spark.read.option("header", "true").csv(CSV_PATH)

    # 简单的清洗：去除空格，转换日期
    for col_name in df.columns:
        df = df.withColumn(col_name, F.trim(F.col(col_name)))

    df = df.withColumn("Date", F.to_date(F.col("Date"), "yyyy-mm-dd"))

    df.write.mode("overwrite").parquet(BRONZE)
    print("✅ Bronze Layer Saved:", BRONZE)
    return df

def build_silver(spark):
    """Bronze -> Silver (Cleaning & Deduplication)"""
    print("🚀 Building Silver Layer...")
    if os.path.exists(SILVER): shutil.rmtree(SILVER)

    df = spark.read.parquet(BRONZE)

    # 过滤无效数据
    df = df.filter(F.col("Date").isNotNull() & F.col("Stock_symbol").isNotNull())
    df = df.dropDuplicates(["Date", "Stock_symbol", "Article_title"])

    # 标准化文本
    df = df.withColumn("Publisher_norm", F.upper(F.trim(F.col("Publisher")))) \
           .withColumn("Author_norm", F.upper(F.trim(F.col("Author"))))

    # 生成唯一 ID (重要：后续 Join 的 Key)
    df = df.withColumn("news_id", F.md5(F.concat_ws("|", "Date", "Stock_symbol", "Article_title")))

    df.write.mode("overwrite").parquet(SILVER)
    print("✅ Silver Layer Saved:", SILVER)
    return df

def build_silver_nlp(spark):
    """Silver -> NLP Ready Data (Prepare text for model)"""
    print("🚀 Preparing Data for NLP...")
    if os.path.exists(NLP_SILVER): shutil.rmtree(NLP_SILVER)

    df = spark.read.parquet(SILVER)

    # 拼接标题和摘要作为模型输入
    nlp_df = df.select(
        "news_id",
        F.concat_ws(" . ", F.col("Article_title"), F.col("Lsa_summary")).alias("text")
    ).filter("text != ''")

    nlp_df.write.mode("overwrite").parquet(NLP_SILVER)
    print("✅ NLP Ready Data Saved:", NLP_SILVER)
    return nlp_df

In [15]:
def build_sentiment_features_scalable(spark):
    """
    【优化】使用 Spark Pandas UDF 执行分布式情感分析
    替代原来的单机循环，性能提升巨大。
    """
    print("🚀 Running Scalable NLP Sentiment Analysis...")
    import torch
    import pandas as pd
    from pyspark.sql.functions import pandas_udf
    from transformers import pipeline

    # 如果已存在则清理
    if os.path.exists(SENTIMENT_OUTPUT): shutil.rmtree(SENTIMENT_OUTPUT)

    # 读取准备好的文本数据
    df = spark.read.parquet(NLP_SILVER)

    # 定义 UDF 输出结构
    schema = StructType([
        StructField("label", StringType(), True),
        StructField("score", DoubleType(), True),
        StructField("score_signed", DoubleType(), True)
    ])

    # 定义 Pandas UDF (将在每个 Worker 节点执行)
    @pandas_udf(schema)
    def sentiment_udf(texts: pd.Series) -> pd.DataFrame:
        # 在 Worker 上加载模型 (建议使用轻量级模型如 distilbert)
        device = 0 if torch.cuda.is_available() else -1
        pipe = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=device)

        # 批量推理
        preds = pipe(texts.tolist(), truncation=True, max_length=256, batch_size=32)

        labels = [p['label'] for p in preds]
        scores = [p['score'] for p in preds]

        # 计算带符号分数
        signed_scores = []
        for l, s in zip(labels, scores):
            signed_scores.append(s if l == 'POSITIVE' else -s)

        return pd.DataFrame({'label': labels, 'score': scores, 'score_signed': signed_scores})

    # 应用 UDF
    result_df = df.withColumn("sentiment", sentiment_udf(F.col("text")))

    # 展开结构体并保存
    final_sentiment = result_df.select(
        "news_id",
        F.col("sentiment.label").alias("sentiment_label"),
        F.col("sentiment.score").alias("sentiment_score"),
        F.col("sentiment.score_signed").alias("sentiment_score_signed")
    )

    final_sentiment.write.mode("overwrite").parquet(SENTIMENT_OUTPUT)
    print("✅ Sentiment Analysis Complete:", SENTIMENT_OUTPUT)
    return final_sentiment

def build_fact_news_sentiment(spark):
    """
    【补全】构建最终事实表 (Fact Table)
    将 Silver 层的明细数据 与 NLP 情感分数 Join 在一起。
    """
    print("🚀 Building FACT_NEWS_SENTIMENT Table...")
    if os.path.exists(FACT_NEWS_SENTIMENT): shutil.rmtree(FACT_NEWS_SENTIMENT)

    # 1. 读取 Silver (业务数据)
    silver_df = spark.read.parquet(SILVER)

    # 2. 读取 Sentiment (情感数据)
    sentiment_df = spark.read.parquet(SENTIMENT_OUTPUT)

    # 3. Join 操作
    fact_df = silver_df.join(sentiment_df, on="news_id", how="left")

    # 4. 选择最终需要的列 (面向分析)
    fact_df = fact_df.select(
        "news_id", "Date", "Stock_symbol", "Article_title", "Url",
        "Publisher_norm", "Author_norm",
        "sentiment_label", "sentiment_score", "sentiment_score_signed"
    )

    fact_df.write.mode("overwrite").parquet(FACT_NEWS_SENTIMENT)
    print("✅ FACT Table Created:", FACT_NEWS_SENTIMENT)
    return fact_df

def build_gold_daily_agg(spark):
    """
    Gold Layer: 每日聚合统计
    """
    print("🚀 Building Gold Aggregation (Daily Stats)...")
    if os.path.exists(GOLD): shutil.rmtree(GOLD)

    df = spark.read.parquet(FACT_NEWS_SENTIMENT) # 从 Fact 表读取更方便

    gold_df = df.groupBy("Date", "Stock_symbol").agg(
        F.count("news_id").alias("news_count"),
        F.avg("sentiment_score_signed").alias("avg_sentiment"),
        F.countDistinct("Publisher_norm").alias("publisher_count")
    )

    gold_df.write.mode("overwrite").parquet(GOLD)
    print("✅ Gold Aggregation Saved:", GOLD)
    return gold_df

In [16]:
def save_to_snowflake(df, table_name):
    """
    将 Spark DataFrame 写入 Snowflake
    需确保已正确配置 spark.jars.packages
    """
    print(f"❄️ Saving to Snowflake table: {table_name}...")

    # 请替换为您的实际配置
    sfOptions = {
        "sfURL": "https://<your_account>.snowflakecomputing.com",
        "sfUser": "<your_username>",
        "sfPassword": "<your_password>",
        "sfDatabase": "YOUR_DB_NAME",
        "sfSchema": "PUBLIC",
        "sfWarehouse": "COMPUTE_WH"
    }

    try:
        df.write \
            .format("net.snowflake.spark.snowflake") \
            .options(**sfOptions) \
            .option("dbtable", table_name) \
            .mode("overwrite") \
            .save()
        print(f"✅ Successfully wrote to {table_name}")
    except Exception as e:
        print(f"❌ Snowflake write failed (Check JAR/Configs): {e}")

In [ ]:
# -------------------------------------------------------------------------
# Main Execution Block
# -------------------------------------------------------------------------

if __name__ == "__main__":
    # 1. 启动 Spark
    spark = create_spark()

    # 2. 基础 ETL
    build_bronze(spark)
    build_silver(spark)

    # 3. NLP 处理 (优化版)
    build_silver_nlp(spark)
    build_sentiment_features_scalable(spark)  # 使用 Pandas UDF 并行处理

    # 4. 构建事实表 (Join)
    fact_df = build_fact_news_sentiment(spark)

    # 5. 构建聚合表 (Gold)
    gold_df = build_gold_daily_agg(spark)

    # 6. 保存到 Snowflake (可选，需填写配置)
    # save_to_snowflake(fact_df, "FACT_NEWS_SENTIMENT")
    # save_to_snowflake(gold_df, "GOLD_DAILY_STATS")

    print("🎉 All ELT Steps Completed Successfully!")

In [4]:
from pyspark.sql import SparkSession

def create_spark(app_name="5400-news-elt"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")            # local CPU
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.files.maxRecordsPerFile", "200000")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.RawLocalFileSystem")
        .config("spark.hadoop.fs.file.impl.disable.cache", "true")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    print("🔥 Spark Ready:", spark.version)
    return spark

In [5]:
import os
from pyspark.sql import functions as F
from pyspark.sql import types as T

BASE = r"D:\Columbia\Fall2025\5400\project\layer\1"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
GOLD = os.path.join(BASE, "gold")
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")


In [6]:
def build_bronze(spark):
    os.makedirs(BRONZE, exist_ok=True)

    print("📥 Loading CSV...")
    df = (
        spark.read.format("csv")
        .options(
            header="true",
            inferSchema="true",
            multiLine="true",
            escape="\"",
            quote="\"",
            mode="PERMISSIVE"
        )
        .load(CSV_PATH)
    )

    # Cleaning string columns
    for c, t in df.dtypes:
        if t == "string":
            df = df.withColumn(c, F.trim(F.col(c)))

    # date
    if "Date" in df.columns:
        df = df.withColumn("Date", F.to_date("Date", "yyyy-MM-dd"))

    df = df.repartition(400)

    print("💾 Writing Bronze...")
    df.write.mode("overwrite").parquet(BRONZE)
    print("🥉 Bronze Ready:", BRONZE)

    return df

def build_silver(spark):
    os.makedirs(SILVER, exist_ok=True)
    df = spark.read.parquet(BRONZE)

    # key fields not null
    if "Date" in df.columns:
        df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns:
        df = df.filter(F.col("Stock_symbol").isNotNull())

    # drop duplicates
    df = df.dropDuplicates()

    # standardize text fields
    if "Publisher" in df.columns:
        df = df.withColumn("Publisher_norm", F.upper("Publisher"))
    if "Author" in df.columns:
        df = df.withColumn("Author_norm", F.upper("Author"))

    # unique ID
    df = df.withColumn("news_id", F.monotonically_increasing_id())

    # repartition
    df = df.repartition(400, "Stock_symbol", "Date")

    print("💾 Writing Silver...")
    df.write.mode("overwrite").parquet(SILVER)
    print("🥈 Silver Ready:", SILVER)

    return df

def build_gold(spark):
    os.makedirs(GOLD, exist_ok=True)
    df = spark.read.parquet(SILVER)

    df = df.withColumn("title_len", F.length("Article_title"))

    agg_cols = [
        F.count("*").alias("article_count"),
        F.countDistinct("Publisher").alias("publisher_count"),
        F.avg("title_len").alias("avg_title_len"),
    ]

    for col in ["Textrank_summary", "Lsa_summary", "Luhn_summary", "Lexrank_summary"]:
        if col in df.columns:
            agg_cols.append(F.first(col, ignorenulls=True).alias("sample_summary"))
            break

    gold_df = df.groupBy("Date", "Stock_symbol").agg(*agg_cols)
    gold_df = gold_df.repartition(200, "Stock_symbol")

    print("💾 Writing Gold...")
    gold_df.write.mode("overwrite").parquet(GOLD)
    print("🥇 Gold Ready:", GOLD)

    return gold_df

In [7]:
spark = create_spark()

bronze_df = build_bronze(spark)
#silver_df = build_silver(spark)
#gold_df   = build_gold(spark)
#nlp_df    = build_silver_nlp(spark)

🔥 Spark Ready: 3.3.1
📥 Loading CSV...
💾 Writing Bronze...
🥉 Bronze Ready: D:\Columbia\Fall2025\5400\project\layer\1\bronze


In [8]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("test")
    .master("local[*]")
    .getOrCreate()
)
